In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

# ## 기본 모델 동결
# Freeze base model
base_model.trainable = False

# ## 모델에 레이어 추가
# Functional Model API
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

# 모델 출력(확인)
model.summary()

# ## 모델 컴파일
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# ## 데이터 증강
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

# ## 데이터세트 로드
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='categorical')

# ## 모델 트레이닝
# 데이터 샘플수, 배치 크기 확인
print(train_it.samples)
print(train_it.batch_size)

hist = model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=5)

# ## 파인튜닝을 위해 모델 동결 해제
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss='categorical_crossentropy' , metrics=['accuracy'])

hist = model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=5)


# ## 모델 평가
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

# ## 평가 실행
from run_assessment import run_assessment

run_assessment(model, valid_it)

# ## 인증서 생성
# 평가에 합격한 경우 아래에 표시된 과정 페이지로 돌아가 "평가 작업" 버튼을 클릭하면 과정 인증서가 생성됩니다.
